# Review: Improved Training of Wasserstein GANs

###### ***Author***

### 1. Contribution

- critic weight clipping의 문제점을 toy dataset을 통해서 보였다.
- gradient penalty를 제안해서 위의 문제를 해결했다.
- 다양한 GAN 구조에 대해 안정적인 학습을 증명하고, weight clipping에 대한 성능 향상, high quality image generation 등이 가능하다.

### 2. Background & Difficulties with weight constraints

#### A. Background


<img style="display: block; margin: auto;"
src="https://user-images.githubusercontent.com/68529301/165379661-bd34fa9b-6e08-4f7a-992c-0808f69839b6.png" alt="2" width="600px" />
<p style="text-align: center;">
Fig1. Proposition1
</p>

<img style="display: block; margin: auto;"
src="https://user-images.githubusercontent.com/68529301/165379678-79569b92-84a8-4fe5-93ad-993d6fbbf4b8.png" alt="2" width="400px" />
<p style="text-align: center;">
Fig2
</p>

<img style="display: block; margin: auto;"
src="https://user-images.githubusercontent.com/68529301/165379682-b07f66d7-5a84-4a28-b526-320ef27d4608.png" alt="2" width="350px" />
<p style="text-align: center;">
Fig3
</p>

<img style="display: block; margin: auto;"
src="https://user-images.githubusercontent.com/68529301/165379684-63010cfa-e135-44bb-a374-8c321f178f39.png" alt="2" width="300px" />
<p style="text-align: center;">
Fig4
</p>

WGAN 손실함수(Fig2)을 max로 만드는 f를 f\*라고 하자. 이 f\*는 Fig3를 통해서 구할 수 있다. 

$x$ \~$P_g$, $y$\~$P_r$로 샘플링을 해서 $x$와 $y$를 보간한 직선 중 $x$와 $y$ 사이의 점 $x_t$가 있다고 할 때, $x_t=tx+(1-t)y$ 로 나타낼 수 있다. (0 ≤ $t$ ≤ 1)

이때, 어떠한 $x_t$이든 Fig4이 만족된다. (증명은 appendix)

<img style="display: block; margin: auto;"
src="https://user-images.githubusercontent.com/68529301/165379686-ea77e61a-74a4-485e-bed3-7c09c058fb1b.png" alt="2" width="500px" />
<p style="text-align: center;">
Fig5
</p>

### B. Weight Clipping의 문제점

- weight clipping: 업데이트된 weight를 특정 구간으로 제한하는 것 → 1-Lipschitz를 만족시키기 위함
- weight clippping이 최적화에 문제를 발생시킨다. optimization이 성공했을때조차 critic이 pathological value surface를 가지는 경우도 있다.

<img style="display: block; margin: auto;"
src="https://user-images.githubusercontent.com/68529301/165379691-dbfc8dbe-2445-4022-9853-de05f036ef98.png" alt="2" width="500px" />
<p style="text-align: center;">
Fig6. 윗줄(weight clipping): 데이터 분포의 higher moments를 잡아내는 데 실패했다. // 밑줄(gradient panelty)
</p>

<img style="display: block; margin: auto;"
src="https://user-images.githubusercontent.com/68529301/165379692-6eec3444-0851-4cc2-bbb4-82f808c8d4dc.png" alt="2" width="300px" />
<p style="text-align: center;">
Fig7. Swiss Roll dataset을 학습시키는 도중 weight clipping을 사용할 때, 발생하는 vanish / explode를 보여주고 gradient penalty를 사용 시 문제가 발생하지 않는 것을 보여주는 gradient norm
</p>

<img style="display: block; margin: auto;"
src="https://user-images.githubusercontent.com/68529301/165379693-389b4474-4854-4bc0-bf5c-8ca20016e014.png" alt="2" width="200px" />
<p style="text-align: center;">
Fig8. weight clipping은 가중치가 clipping boundary 근처로 몰리지만 gradient panelty는 그렇지 않는 것을 알 수 있다.
</p>

# 3. Gradient Panelty

## A. gradient panelty

<img style="display: block; margin: auto;"
src="https://user-images.githubusercontent.com/68529301/165379696-b8bcb12b-4adf-44be-8ce5-b4759baa562b.png" alt="2" width="600px" />
<p style="text-align: center;">
Fig9
</p>

- gradient의 norm이 항상 1보다 작으면, 미분 가능한 함수는 1-Libschtiz를 만족한다.
- 이 내용을 사용해서, critic의 output의 gradient norm을 제한함.

## B. Sampling distribution

- 데이터 분포 $P_r$과 generator 분포 $P_g$에서 뽑은 sample 사이를 이은 직선에서 $\hat{x}$을 sampling하여 사용하였다.
- 이는 최적의 critic이 $P_r, P_g$로부터의 점들을 연결하는 gradient norm 1의 직선을 가지고 있다는 점에서 motivation을 받았다.
- 모든 곳에서 gradient norm constraint를 주는 것은 어렵기 때문에, 이렇게 직선을 따라 하는 것도 실험적으로 좋은 성능을 보였다.

## C. Penalty coefficient

- 전체 실험에 대해서 $\lambda$=10을 사용함. (toy task, ImageNet CNN까지 잘 된다)

## D. No critic batch normalization

- 대부분의 GAN은 batch normalization을 discriminator와 generator 둘다에 적용해서 학습을 안정화시키려했지만, batch normalization은 discriminator의 단일 입력을 단일 출력으로 매핑하는 문제로부터, 입력의 전체 배치로부터 출력의 배치로 매핑하는 문제로 유형을 변화시킨다.
- 기존처럼 전체 batch를 penalize하는 것이라 아니라 각 input에 독립적으로 critic의 gradient norm을 penalize 시키기 때문에, batch normalization은 wgan gp에 맞지 않는다.
- 그래서 batch normalizaiton을 생략하고 훈련을 진행하였으며, 또한 batch normalization보다는 layer normalization을 사용하는 것을 추천한다.

## E. Two-sided penalty

- **One-sided penalty**
    
<img style="display: block; margin: auto;"
src="https://user-images.githubusercontent.com/68529301/165379699-94b95fee-aafd-449e-84a1-a49fb765c946.png" alt="2" width="600px" />
<p style="text-align: center;">
Fig10
</p>
    
- **Two-sided penalty**
    
<img style="display: block; margin: auto;"
src="https://user-images.githubusercontent.com/68529301/165379701-a094b806-ca24-4ff2-9325-fe7019ba8816.png" alt="2" width="600px" />
<p style="text-align: center;">
Fig11
</p>
    
- gradient norm이 1 이하가 되도록 (one-sided penalty)하는 것보다 1로 향하도록 (two-sided penalty)를 촉진한다.
- one-sided penalty가 더 좋은 경우도 있고 two-sided penalty가 더 좋은 경우도 있다.
- 그럼에도 two-sided penalty를 사용한 이유는 언급되어 있지 않고 후속 연구로 넘겨버렸다.

# 4. Experiments

## A. training random architectures within a set

- 200개의 아키텍처를 standard GAN, WGAN-GP로 각각 학습시키고 점수를 비교함
- 기준: inception_score > min_score
    
<img style="display: block; margin: auto;"
src="https://user-images.githubusercontent.com/68529301/165379703-f7fbea91-3881-4857-ad5e-01bcbcfcca1a.png" alt="2" width="600px" />
<p style="text-align: center;">
Fig12
</p>
    

## B. LSUN bedroom dataset

- WGAN-GP는 batch normalization 대신 layer normalization을 사용하였다.
- WGAN-GP를 제외한 모델들은 불안정하거나 mode collapse에 빠진 모습을 볼 수 있다.
    
<img style="display: block; margin: auto;"
src="https://user-images.githubusercontent.com/68529301/165379708-6fe0f980-6dec-4f10-a4c1-3218757bdeac.png" alt="2" width="600px" />
<p style="text-align: center;">
Fig13
</p>
    
    

## C. Improved performance over weight clipping

- weight clipping보다 학습 속도가 빠르고 sample quality가 좋아졌음을 증명
- 아래의 plot을 보면 weight clipping 보다 converge 속도도 빠르고 성능도 좋다는 것을 알 수 있다.
- weight clipping은 RMSProp optimizer를 사용하였다.
- WGAN-GP에 Adam optimizer를 적용했는데 성능이 더 좋았다.
- DCGAN보다 속도는 느리지만 converge 되었을때 score가 더 안정적이다.
    
<img style="display: block; margin: auto;"
src="https://user-images.githubusercontent.com/68529301/165379712-fd94f51f-6daa-4bd1-8856-7ece6fb09432.png" alt="2" width="600px" />
<p style="text-align: center;">
Fig14
</p>
    

## D. Sample quality on CIFAR-10 and LSUN bedrooms

- unsupervised model은  SOTA, supervised model은 SGAN을 제외한 다른 GAN들보다 성능이 우수했다.
    
<img style="display: block; margin: auto;"
src="https://user-images.githubusercontent.com/68529301/165379715-e203f98d-5b97-45c4-a0a7-ea18461a7d0b.png" alt="2" width="600px" />
<p style="text-align: center;">
Fig15
</p>

    

## E. Modeling discrete data with a continuous generator

## F. meaningful loss curves and detecting overfitting

<img style="display: block; margin: auto;"
src="https://user-images.githubusercontent.com/68529301/165379718-88870f71-72a2-4b4e-918c-a2b0d290131e.png" alt="2" width="600px" />
<p style="text-align: center;">
Fig16
</p>

- weight clipping의 장점이 loss가 sample quality와 상관 관계가 있고 minimum으로 수렴한다는 데에 있다.
- gradient panelty도 plot을 확인했을 때, genreator가 $W(P_r, P_g)$를 최소화하면서 loss가 수렴한다는 것을 알 수 있다.
- WGAN-GP는 critic에서의 과적합을 탐지하고 네트워크가 최소화하는 동일한 loss에 대해 과적합을 측정한다.

# 5. Implementation

<img style="display: block; margin: auto;"
src="https://user-images.githubusercontent.com/68529301/165379721-5848cd13-e363-433f-a3bc-6cebd2b1216b.png" alt="2" width="600px" />
<p style="text-align: center;">
Fig17
</p>

In [ ]:
def compute_gradient_penalty(D, real_samples, fake_samples):
    """Calculates the gradient penalty loss for WGAN GP"""

    # Random weight term for interpolation between real and fake samples
    alpha = Tensor(np.random.random((real_samples.size(0), 1, 1, 1)))

    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)
    d_interpolates = D(interpolates)
    fake = Variable(Tensor(real_samples.shape[0], 1).fill_(1.0), requires_grad=False)

    # Get gradient w.r.t. interpolates
    gradients = autograd.grad(
        outputs=d_interpolates,
        inputs=interpolates,
        grad_outputs=fake,
        create_graph=True,
        retain_graph=True,
        only_inputs=True,
    )[0]

    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()

    return gradient_penalty